This notebook implements FFH estimation using:
* feature detection results from streetview images
* elevation, depth and classification rasters of building facades derived from lidar data.

## Load modules

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import os
import glob
from PIL import Image
# from GSV.geometry import extract_feature_pixels_lowest_region, calculate_height_difference, calculate_width_difference, estimate_FFH, estimate_FFE
from lidar.point_cloud_processings import estimate_FFH, get_closest_ground_to_feature,select_best_feature, compute_feature_properties, calculate_gapfill_depth
import geopandas as gpd
# import matplotlib.pyplot as plt
# from sklearn.metrics import root_mean_squared_error, mean_absolute_error
import pandas as pd
import numpy as np

## Set input parameters
* building points containing ground elevation data
* object detection results from streetview panoramas, containing buildnig ID and panorama ID
* folder containing building facade elevation and classifcation rasters

In [ ]:
building_points_file=r'/home/ubuntu/lavender_floor_height/output/Final_Wagga_training_samples_pano_metadata_clipping_elevations.geojson'
predictions_file=r'/mnt/floorheightvolume/lidar_Wagga/yolo_predictions/object_detection_results.csv'
las_projections_folder=r"/mnt/floorheightvolume/lidar_Wagga/clipped_projected/"
out_buildings_file=r'/home/ubuntu/lavender_floor_height/output/Final_Wagga_training_samples_pano_metadata_clipping_elevations_ffhs.geojson'

## Read feature detection results

In [ ]:
df_predictions = pd.read_csv(predictions_file)
df_predictions.head()

## Read in building points

In [ ]:
gdf_building_points=gpd.read_file(building_points_file)
gdf_building_points=gdf_building_points[gdf_building_points["USAGE"]=="Residential"].reset_index(drop=True)
gdf_building_points.head()

In [ ]:
gdf_building_points.columns

## Test calculation with one example

In [ ]:
i=0
ufi=gdf_building_points.loc[i,'UFI']
ufi

Identify prediction results by building UFI:

In [ ]:
formatted_search = f"_{ufi}"
df_single=df_predictions[df_predictions['pano_id'].str.endswith(formatted_search)].reset_index(drop=True)
df_single

In [ ]:
ground_elevation_gapfill = gdf_building_points.iloc[i]['lidar_elev_25pct']
ground_elevation_gapfill

#### Read in corresponding projected rasters
* Elevation
* classification
* Depth

In [ ]:
elevation_file=glob.glob(os.path.join(las_projections_folder,'*'+'_UFI_'+str(ufi)+'_elevation_resampled.tif'))[0]
classification_file=glob.glob(os.path.join(las_projections_folder,'*'+'_UFI_'+str(ufi)+'_classification_resampled.tif'))[0]
depth_file=glob.glob(os.path.join(las_projections_folder,'*'+'_UFI_'+str(ufi)+'_depth_resampled.tif'))[0]

In [ ]:
classification_arr=np.array(Image.open(classification_file))
elevation_arr=np.array(Image.open(elevation_file))
depth_arr=np.array(Image.open(depth_file))

### Calculate gapfilling depth

In [ ]:
gapfill_depth=calculate_gapfill_depth(depth_arr, classification_arr,nodata_depth=9999)
gapfill_depth

### Calculate feature properties
* metrics for filtering: width, height, area and width/heigh ratio

In [ ]:
for j in range(len(df_single)):
    df_single.loc[j,['top_elevation','bottom_elevation','width_m', 'height_m','area_m2','ratio']] = compute_feature_properties(row=df_single.iloc[j],elevation_arr=elevation_arr,depth_arr=depth_arr,gapfill_depth=gapfill_depth)
df_single

### Filter detected features when multiple features are detected
* Keep the Front Door with closest to standard feature metrics
* Keep the lowest and most confident feature for the other classes

In [ ]:
# Define real-world target values for Australian Front Doors
frontdoor_standards = {
    "width_m": 0.82,      # meters
    "height_m": 2.04,     # meters
    "area_m2": 1.67,      # meters²
    "ratio": 0.40         # width-to-height ratio
}
weights = {'area_m2': 1, 'ratio': 1, 'confidence':1, 'x_location':1, 'y_location': 1}  # Default weights
classes=["Foundation", "Front Door", "Garage Door", "Stairs"]

In [ ]:
df_selected=select_best_feature(df_single, weights=weights, classes=classes, 
                                img_width=depth_arr.shape[1], img_height=depth_arr.shape[0],
                                frontdoor_standards=frontdoor_standards).reset_index(drop=True)
df_selected

#### Calculate FFHs based on rules

In [ ]:
for j in range(len(df_selected)):
    df_selected.loc[j,['nearest_ground_elev']] = get_closest_ground_to_feature(row=df_selected.iloc[j],classification_arr=classification_arr,elevation_arr=elevation_arr,min_area=5)
df_selected

In [ ]:
FFH1, FFH2, FFH3 = estimate_FFH(df_selected, ground_elevation_gapfill)
FFH1, FFH2, FFH3

### Attach results back to building points

In [ ]:
gdf_building_points_updated=gdf_building_points.copy()
gdf_building_points_updated.loc[i,['FFH1','FFH2','FFH3']]=[FFH1, FFH2, FFH3]

## Batch calculation for all buildings

In [ ]:
for i in range(len(gdf_building_points)):
    try:
        # get building ufi
        ufi=gdf_building_points.loc[i,'UFI']
        print('processing building ',ufi)

        # find predictions
        formatted_search = f"_{ufi}"
        df_single=df_predictions[df_predictions['pano_id'].str.endswith(formatted_search)].reset_index(drop=True)
        # print('features before filtering: ',df_single['class'].values.tolist())
        # print('features confidence: ',df_single['confidence'].values.tolist())
        
        # extract elevation around house
        ground_elevation_gapfill = gdf_building_points.iloc[i]['lidar_elev_25pct']
        print('ground elevation around house: ',ground_elevation_gapfill)
        
        # find and read lidar projected rasters
        elevation_file=glob.glob(os.path.join(las_projections_folder,'*'+'_UFI_'+str(ufi)+'_elevation_resampled.tif'))[0]
        classification_file=glob.glob(os.path.join(las_projections_folder,'*'+'_UFI_'+str(ufi)+'_classification_resampled.tif'))[0]
        depth_file=glob.glob(os.path.join(las_projections_folder,'*'+'_UFI_'+str(ufi)+'_depth_resampled.tif'))[0]
        classification_arr=np.array(Image.open(classification_file))
        elevation_arr=np.array(Image.open(elevation_file))
        depth_arr=np.array(Image.open(depth_file))
        
        # calculate gapfilling depth
        gapfill_depth=calculate_gapfill_depth(depth_arr, classification_arr,nodata_depth=9999)
        print('gapfill_depth (m): ',gapfill_depth)

        # calculate feature metrics
        for j in range(len(df_single)):
            df_single.loc[j,['top_elevation','bottom_elevation','width_m', 'height_m','area_m2','ratio']] = compute_feature_properties(row=df_single.iloc[j],elevation_arr=elevation_arr,depth_arr=depth_arr,gapfill_depth=gapfill_depth)
        
        # filter features
        df_selected=select_best_feature(df_single, weights=weights, classes=classes, img_width=depth_arr.shape[1], 
                                        img_height=depth_arr.shape[0], frontdoor_standards=frontdoor_standards).reset_index(drop=True)
        print('features after filtering: ',df_selected['class'].values.tolist())
        print('features confidence: ',df_selected['confidence'].values.tolist())

        # get elevation of nearest ground to feature
        for j in range(len(df_selected)):
            df_selected.loc[j,['nearest_ground_elev']] = get_closest_ground_to_feature(row=df_selected.iloc[j],classification_arr=classification_arr,elevation_arr=elevation_arr,min_area=5)

        # calculate FFHs
        FFH1, FFH2, FFH3 = estimate_FFH(df_selected, ground_elevation_gapfill=ground_elevation_gapfill)
        print('FFHs: ', FFH1, FFH2, FFH3)

        # attach back to building points
        gdf_building_points_updated.loc[i,['FFH1','FFH2','FFH3']]=[FFH1, FFH2, FFH3]
    except Exception as e:
        print(e)

In [ ]:
gdf_building_points_updated.to_file(out_buildings_file)